# MorphCT Example Workflow

1. Start with an atomistic snapshot
2. Determine which atom indices belong to which chromophore.
3. Calculate the energies for each chromophore and chromophore pair using quantum chemical calculations (QCC)
4. Run the kinetic monte carlo (KMC) algorithm to calculate charge mobility

First let's import necessary modules:

In [1]:
import numpy as np
import re
from morphct.system import System
from morphct.chromophores import amber_dict
import gsd.hoomd

def vmd_index_slicer(filepath):
    with open(filepath, "r") as f:
        index_ids = f.readlines()
    numbers = []
    
    for line in index_ids:
        thing = re.findall("\/([0-9]+)(?=[^\/]*$)", line)
        try:
            numbers.append(int(thing[0]))
        except:
            pass
        
    indices=[]
    
    for i in numbers:
        if i not in indices:
            indices.append(i)
            
    return indices


In the cell below, we'll create a system object for morphct. This the main class that will hold all the information for our simulation. We'll need to give it a gsd file, path to an output directory, the frame of the gsd file to use, the scaling factor to convert the lengths in the gsd to Angstroms, and a dictionary to map particle types to elements. Here's our starting structure, an atomistic (not coarse-grain or united atom) gsd file with 10 ITIC molecules:

In [2]:
gsd_file = "/home/jimmy/repos/morphct-jimmy/examples/PEKK_post_pull.gsd"
with gsd.hoomd.open(name=gsd_file, mode='rb') as f:
    snap = f[0]

system = System(gsd_file, "output_pekk_4", frame=1, scale=3.5636, conversion_dict=amber_dict)
system.visualize_system()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
pekk_inds = vmd_index_slicer("/home/jimmy/repos/uli-init/pekk_ids.txt")
pekk_inds = np.array(pekk_inds)
middle_list =[]
print(pekk_inds)

for i in range(len(pekk_inds)):
    pekk_inds[i] = pekk_inds[i]%352
pekk_inds.sort()
middle_list.append(pekk_inds)

##for i in range(24):
##    master_list.append(pekk_inds)
##    pekk_inds = pekk_inds +36
print(len(pekk_inds))

[755 754 753 756 768 757 769 750 751 766 752 767 748 749 747 758 770 759
 771 746 765 745 764 744 743 742 760 772 761 773 739 740 762 741 763]
35


In [4]:
pekk_1 = pekk_inds + 35
middle_list.append(pekk_1)

In [5]:
pekk_2 = pekk_1 + 35
middle_list.append(pekk_2)

In [6]:
pekk_3 = pekk_2 + 35
middle_list.append(pekk_3)

In [7]:
pekk_4 = pekk_3 + 35
middle_list.append(pekk_4)

In [8]:
pekk_5 = pekk_4 + 35
middle_list.append(pekk_5)

In [9]:
pekk_6 = pekk_5 + 35
middle_list.append(pekk_6)

In [10]:
pekk_7 = pekk_6 + 35
middle_list.append(pekk_7)

In [11]:
master_list =[]
from morphct.mobility_kmc import snap_molecule_indices
gsd_mol_index = snap_molecule_indices(snap)
k = np.count_nonzero(gsd_mol_index==0)

pekk_front = np.array(list(range(36-1)))
pekk_front = np.append(pekk_front, k-2)
print(pekk_front)
master_list.append(pekk_front)
print(len(pekk_front))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34 350]
36


In [12]:

pekk_back =list(range(315,352))
pekk_back.remove(350)
pekk_back = np.array(pekk_back)
print(len(pekk_back))
master_list.append(pekk_back)
print(master_list)

36
[array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34, 350]), array([315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327,
       328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340,
       341, 342, 343, 344, 345, 346, 347, 348, 349, 351])]


In [13]:
#master_list =[]
#pekk_inds = np.array(list(range(0,34)))
#for i in range(5):
#    master_list.append(pekk_inds)
#    pekk_inds = pekk_inds +180
#

need to extrapolate these indeces to the rest of the morph below

In [14]:
for i in range(len(middle_list)):
    for x in range(1,10):
        middle_list.append(middle_list[i] +352*x)
for i in range(len(master_list)):
    for x in range(1,10):
        master_list.append(master_list[i] +352*x)
   


In [15]:
np.savetxt("pekk_ids.csv", pekk_inds, fmt="%i")
np.savetxt("pekk_front.csv", pekk_front, fmt="%i")
np.savetxt("pekk_back.csv", pekk_back, fmt="%i")

In [16]:
system.add_chromophores(
    middle_list, "donor")
system.add_chromophores(
    master_list, "donor")

system.visualize_chromophores()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [17]:
%time
system.compute_energies()

There are 766 chromophore pairs
Starting singles energy calculation...
Finished in 29.89 s. Output written to output_pekk_4/singles_energies.txt.
Starting dimer energy calculation...
Finished in 457.03 s. Output written to output_pekk_4/dimer_energies.txt.


In [18]:
system.set_energies()

Energies set.


This function sets the `homo_1`, `homo`, `lumo`, `lumo_1`, `neighbors_delta_e`, and `neighbors_ti` attribute of each chromphore:

In [19]:
i = 0
chromo = system.chromophores[i]
print(f"Chromophore {i}:")
print(
    f"HOMO-1: {chromo.homo_1:.2f} HOMO: {chromo.homo:.2f} LUMO: {chromo.lumo:.2f} "
    f"LUMO+1: {chromo.lumo_1:.2f}"
)
print(f"{len(chromo.neighbors)} neighbors")
print(f"DeltaE of first neighbor: {chromo.neighbors_delta_e[0]:.3f}")
print(f"Transfer integral of first neighbor: {chromo.neighbors_ti[0]:.3f}")

Chromophore 0:
HOMO-1: -8.56 HOMO: -8.40 LUMO: -1.02 LUMO+1: -0.03
11 neighbors
DeltaE of first neighbor: -0.197
Transfer integral of first neighbor: 0.054


With all the energy values set, we're ready to run KMC! We need to set the temperature that the KMC simulation will be run at and the lifetimes and numbers of our carriers:

In [20]:
from morphct.mobility_kmc import get_jobslist
jobs = get_jobslist([1.00e-12], 10)
import multiprocessing as mp
mp.cpu_count()
print(jobs)

[[(5, 1e-12, 'hole')], [(1, 1e-12, 'hole')], [(8, 1e-12, 'hole')], [(7, 1e-12, 'hole')], [(2, 1e-12, 'hole')], [(6, 1e-12, 'hole')], [(3, 1e-12, 'hole')], [(9, 1e-12, 'hole')], [(0, 1e-12, 'hole')], [(4, 1e-12, 'hole')]]


In [21]:
print(len(jobs))

10


In [22]:
%%time
lifetimes = [1e-10,1e-9, 1e-8]
temp = 300
system.run_kmc(lifetimes, temp, n_holes=500, verbose=1)

All KMC jobs completed!
Combining outputs...
---------- KMC_ANALYZE ----------
All figures saved in output_pekk_4/kmc/figures
---------------------------------
Considering the transport of hole...
Obtaining mean squared displacements...
Plotting distribution of hole displacements
	Figure saved as hole_displacement_dist.png
Calculating mobility...
	Standard Error 3.4397789704110597e-10
	Fitting r_val = 0.9999999545775525
	Figure saved as lin_MSD_hole.png
	Figure saved as semi_log_MSD_hole.png
	Figure saved as log_MSD_hole.png
	----------------------------------------
	Hole mobility = 7.36E-02  +/- 4.95E-05 cm^2 V^-1 s^-1
	----------------------------------------
Calculating hole trajectory anisotropy...
	----------------------------------------
	Hole charge transport anisotropy: 0.076
	----------------------------------------
Plotting hole hop frequency distribution...
	DYNAMIC CUT
	Notice: No minima found in distribution. Cutoff set to None.
	Cluster cut-off based on hop frequency set 

/home/jimmy/miniconda3/envs/morphct-ex/lib/python3.7/site-packages/scipy/optimize/minpack.py:834: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


	Notice: No minima found in distribution. Cutoff set to None.
Neighbor histogram figure saved as neighbor_hist_donor.png
	Notice: No minima found in distribution. Cutoff set to None.
Orientation histogram figure saved as orientation_hist_donor.png
	Notice: No minima found in distribution. Cutoff set to None.
	Figure saved as donor_transfer_integral_mols.png
Examining the donor material...
Calculating clusters...
	No cutoff provided: cluster cutoff set to 9.686
	----------------------------------------
	Donor: Detected 1 total
	and 1 large clusters (size > 6).
	Largest cluster size: 100 chromophores.
	Ratio in "large" clusters: 1.00
	----------------------------------------
Examining the acceptor material...
	No material found. Continuing...
Mean intra-cluster donor rate: 3.089e+13+/-2.832e+11
	Figure saved as donor_hopping_rate_clusters.png
	Figure saved as donor_transfer_integral_clusters.png
Mean intra-molecular donor rate: 2.945e+13+/-6.426e+11
Mean inter-molecular donor rate: 3.150

with a p.join(1000) it took 4 hours. lets see how long it takes with p.join(1) (output_pekk_1)
we get the same mobiliy but it took 15 minutes and we still got 60 displacemnets so its not like it 
killed the process (output_pekk_3)
tool the lifes down an order of magnitiude each to 10^-10 and 10^-8 and now it takes 1 minute. 


In [34]:
carrier_data= system._carrier_data
print(system._carrier_data["displacement"])

[1086.2347351594988, 972.2376555262003, 108.46136814138282, 131.23535539383636, 113.27963368675034, 170.32421265917714, 192.37961757726464, 262.8125323077377, 1424.9884559016689, 47.68487481839808, 727.8929457421999, 975.1764173046317, 423.39296072896525, 957.1600436226013, 59.28635084082771, 1309.2613074430003, 911.8016819546526, 240.0687715773573, 101.57117164086348, 195.83259441536566, 731.1754889737388, 924.8107759689527, 1092.2597012282763, 183.1426263822995, 727.9159385752469, 74.82109885780842, 1367.55011287314, 694.9568356263817, 897.2057791393289, 344.6716504817315, 407.55500658534754, 203.21947333732584, 376.9537367350442, 799.3020193729614, 47.930705612025754, 68.97736553043796, 82.5026360201496, 279.3314733884785, 1333.0469925337648, 41.80533024521936, 461.93124134460226, 148.5480631673825, 581.5707587842329, 107.04136174745973, 615.314979994567, 62.59090477860072, 39.49650585405215, 34.33426381778046, 285.52664416994327, 384.18652153511783, 378.8515939393388, 197.095239139

5 holes gave 7.2 seconds wall time
8 holes gave 7.45 seconds wall time
10 holes gave 7.81 seconds


In [24]:
from morphct.kmc_analyze import get_times_msds
get_times_msds(system._carrier_data)

([1e-08, 1e-10, 1e-09],
 [1.1415599220180277e-14, 1.1884667760501344e-16, 1.142216960830171e-15],
 [2.032712802778919e-15, 2.689860270418506e-15, 2.249897830821804e-15],
 [2.059851690474096e-17, 2.1719091260110215e-19, 1.987276572727949e-18])

In [35]:
print(len(carrier_data["lifetime"]))

1500


In [33]:
print(carrier_data["displacement"][1])

972.2376555262003


In [27]:
from collections import defaultdict
total = 0
total_averaged = 0
squared_disps = defaultdict(list)
actual_times = defaultdict(list)
for i, displacement in enumerate(carrier_data["displacement"]):
    current = carrier_data["current_time"][i]
    lt = carrier_data["lifetime"][i]
    n_hops = carrier_data["n_hops"][i]
    if current > lt * 2 or current < lt / 2 or n_hops == 1:
        total += 1
        continue
    # A -> m
    squared_disps[lt] += [(carrier_data["displacement"][i] * 1e-10) ** 2]
    actual_times[lt] += [current]

    # Also keep track of whether each carrier is a hole or an electron
    total_averaged += 1
    total += 1
if total > total_averaged:
    print(f"\tNotice: The data from {total - total_averaged} carriers were")
    print("\tdiscarded due to the carrier lifetime being more than double")
    print("\t(or less than half of) the specified carrier lifetime.")
times = []
msds = []
time_stderr = []
msd_stderr = []
for lt, disps in squared_disps.items():
    times.append(lt)
    time_stderr.append(np.std(actual_times[lt]) / len(actual_times[lt]))
    msds.append(np.average(disps))
    msd_stderr.append(np.std(disps) / len(disps))
    

In [28]:
print(squared_disps.items())

dict_items([(1e-08, [1.1799058998670266e-14, 9.452460588230827e-15, 3.70099172591746e-16, 2.0305920994530228e-14, 9.509690448670974e-15, 9.1615534910762e-15, 1.7141651711673547e-14, 8.313823072153336e-15, 5.346175956759861e-15, 8.552749713482964e-15, 1.1930312549272836e-14, 3.3541221598206547e-16, 5.298616136318827e-15, 1.8701933112193386e-14, 4.8296500338383375e-15, 8.049782101210102e-15, 1.1879854664580088e-15, 1.6610108339278268e-15, 1.4209411963851302e-15, 6.38883718173694e-15, 1.7770142843033147e-14, 2.133804717301652e-15, 3.382245474728684e-15, 3.786125246057144e-15, 1.1627344141589629e-14, 9.337614163472105e-15, 2.8376467651059808e-15, 2.1537208890497824e-16, 1.0853036478486174e-14, 4.923444599772731e-16, 4.15368953338121e-14, 5.440251738436237e-15, 3.327671074850248e-15, 8.745632132897394e-15, 7.841951837830584e-15, 1.4740301784736762e-15, 3.934202383335674e-14, 1.2638218516998301e-14, 6.3920719097986356e-15, 1.4760611057462934e-14, 3.464979881790826e-15, 7.463433969479531e-15,

In [29]:
print(msds)

[1.1415599220180277e-14, 1.1884667760501344e-16, 1.142216960830171e-15]
